# 참고2. KoGPT 모델 추론하기

- 학습 내용
    - Hard Prompt 방식을 활용하여 GPT 모델에게 생성 요약을 시킵니다.
    - [8-bit Matrix Multiplication](https://huggingface.co/blog/hf-bitsandbytes-integration) 를 활용하여 큰 언어모델을 작은 GPU 메모리에서 실행합니다.
- 환경 필요사항
    - KaKao-Brain에서 공개한 Ko-GPT3 모델을 활용
    - 일종의 경량화 버전인 float-16 버전의 모델을 활용하여도 Colab 무료 사용으로는 실습 제한
    - 원활한 실습을 위해서는 월 9.99 달러의 Colab Pro를 활용하는것을 권장
<!-- ![nn](prompt-to-generate.jpg) -->

In [1]:
# 구글 드라이브와 연동합니다. 권한 허용이 필요합니다.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 라이브러리를 설치합니다. bitsandbytes는 load_in_8bit옵션을 사용하려면 설치되어야합니다.
%pip install -q transformers accelerate
%pip install -q bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 20.1 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader,Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM 
from tqdm.auto import tqdm
import os, gc
from datetime import datetime, timezone, timedelta

## 모델, 토크나이저 불러오기

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',
)

- load_in_8bit 옵션을 사용하면 [bitsandbytes](https://github.com/TimDettmers/bitsandbytes)를 이용한 8bit 연산 방식으로 불러옵니다.
- 불러올 모델은 float16을 사용하므로 dtype을 지정해주면 불러오는 과정에 사용하는 메모리를 절약할 수 있습니다.
- device_map = 'auto'를 사용하여 weight를 불러오면서 바로 GPU로 올리면 CPU 메모리가 작아도 실행 할 수 있습니다.

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision = 'KoGPT6B-ryan1.5b-float16',
    torch_dtype = torch.float16,
    load_in_8bit = True,
    device_map = 'auto',
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-hm-3je75apctkaun --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true'), PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=

In [7]:
pd.DataFrame([
    (param.dtype, param.shape, param.device, param.requires_grad, name)
    for name, param in model.named_parameters()
], columns=['dtype', 'shape', 'device', 'requires_grad', 'name'])

,dtype,shape,device,requires_grad,name
0,torch.float16,"(64512, 4096)",cuda:0,True,transformer.wte.weight
1,torch.float16,"(4096,)",cuda:0,True,transformer.h.0.ln_1.weight
2,torch.float16,"(4096,)",cuda:0,True,transformer.h.0.ln_1.bias
3,torch.int8,"(4096, 4096)",cuda:0,False,transformer.h.0.attn.k_proj.weight
4,torch.int8,"(4096, 4096)",cuda:0,False,transformer.h.0.attn.v_proj.weight
...,...,...,...,...,...
280,torch.float16,"(4096,)",cuda:0,True,transformer.h.27.mlp.fc_out.bias
281,torch.float16,"(4096,)",cuda:0,True,transformer.ln_f.weight
282,torch.float16,"(4096,)",cuda:0,True,transformer.ln_f.bias
283,torch.float16,"(64512, 4096)",cuda:0,True,lm_head.weight


## Hard Prompt 데이터 셋 작성
- `{본문} 한줄 요약:` 형태로 input을 만들고 모델이 뒷부분을 생성하게 합니다.
- 모델은 따로 학습하지 않으므로 `한줄 요약:`이라는 prompt를 통해서만 요약과제인 것을 알 수 있습니다.
- 길이가 서로 다른 샘플을 하나의 배치로 만들기 위해 collate함수를 작성합니다.
- GPTJ는 문장의 오른쪽 끝부터 생성하는 autoregressive 모델이므로 오른쪽 끝이 같아야합니다.
- 배치 내에서 가장 긴 샘플에 맞춰서 left padding 합니다. 즉, `[PAD][PAD]...[PAD] {본문} 한줄 요약:` 형태가 됩니다.
- `[PAD]`는 정보를 담고 있지 않으므로 attention_mask를 0으로 설정해서 무시합니다.

In [8]:
class SummaryTestDataset(Dataset):
    def __init__(self, data_path, tokenizer):
        self._data = pd.read_csv(data_path)
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self._data)
    
    def __getitem__(self, idx):
        row = self._data.iloc[idx]
        prompt = "{text} 한줄 요약:"
        input_text = prompt.format(text=row['text'])
        input_encoding = self.tokenizer(input_text)

        result = {
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
        }
        
        return result

    def _left_pad(self, sequence, value, max_len):
        return [value] * (max_len - len(sequence)) + sequence

    def collate_fn(self, batch, device='cuda'):
        input_length = max(len(row['input_ids']) for row in batch)

        input_ids = [
            self._left_pad(row['input_ids'], self.tokenizer.pad_token_id, input_length)
            for row in batch
        ]
        attention_mask = [
            self._left_pad(row['attention_mask'], 0, input_length)
            for row in batch
        ]

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long, device=device),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long, device=device),
        }

In [10]:
test_path = '/content/drive/MyDrive/공모전/GPT_Competition/data/test.csv'
test_set = SummaryTestDataset(test_path, tokenizer)
test_loader = DataLoader(test_set, batch_size=4, num_workers=0, shuffle=False, collate_fn=test_set.collate_fn)

## 생성 요약하기

- [generate](https://huggingface.co/docs/transformers/v4.27.1/en/main_classes/text_generation#transformers.GenerationMixin.generate) 메소드로 이어질 문장을 생성합니다.
- colab 기준 1시간 정도 소요됩니다.
- generate 생성방법에 대한 설명은 커뮤니티를 참조하세요.
    - max_length: (프롬프트 포함) 문장 최대 토큰 수
    - max_new_tokens: 생성할 최대 토큰 수
    - min_new tokens: 생성할 최소 토큰 수
    - num_beams: beam_search에 사용할 beam 수 입니다. beam search 하지 않는 경우 1입니다.
    - do_sample: 랜덤 샘플합니다. False면 좋은 값만 선택합니다.
    - temperature: 높을수록 더 기존 확률을 무시하고 샘플합니다.
    - top_k: 각 토큰을 확률이 높은 k개 중에서만 선택합니다.
    - top_p: 각 토큰을 확률순으로 누적합이 p 이하인 범위에서만 선택합니다.
    - [GenerationConfig](https://huggingface.co/docs/transformers/v4.27.1/en/main_classes/text_generation#transformers.GenerationConfig)에서 더 확인 할 수 있습니다.

In [ ]:
def predict():
    preds = []
    model.eval()
    for batch_idx, batch in enumerate(tqdm(test_loader)):
        with torch.no_grad():
            with torch.amp.autocast('cuda'):
                generated = model.generate(
                    input_ids = batch['input_ids'],
                    attention_mask = batch['attention_mask'],

                    pad_token_id = tokenizer.pad_token_id,
                    max_new_tokens = 100,
                    do_sample = False,
                    num_beams = 1,
                    num_beam_groups = 1,
                    use_cache = True,
                )
            prompted_length = batch['input_ids'].size(-1)
            summary_tokens = generated[:, prompted_length:]
            summary = tokenizer.batch_decode(summary_tokens, skip_special_tokens=True)
            preds.extend(summary)
            print(*summary, sep='\n----------\n',end='\n========\n')
    return preds

preds = predict()

  0%|          | 0/125 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


 어린이집·유치원·초등학교에서 아이들이 일상적으로 듣는 말이다.
   서울시여성가족재단은 지난 20일 세계 어린이날을 맞아 보육·교육기관에서 어린이가 겪는 성차별적 말과 행동을 양성평등 관점에서 바꾼 성평등 어린이사전을 발표했다.
   1053명의 시민이 참여해 1406건의 개선안을 제안했다.
   시민 대상 설문조사에서 어린이가 겪는 성차별이 가장 심한 부분은 교
----------
 정부가 코로나19 여파로 직격탄을 맞은 사회적 취약계층과 소상공인 등에 대해 한시적으로 전기요금 납부를 유예해주는 방안을 검토중이다.
----------
 창원 경상대병원에서 간호사들이 의사로부터 수년간 폭언과 폭행을 당했다는 진정이 고용노동부에 접수됐다.
----------
 클렌즈 주스는 건강을 위해 마시는 것이 아니라, 다이어트를 위해 마시는 것이다.
 수능 연기 고충처리센터 운영, 대입전형이 종료되는 2018년 2월 28일까지 운영.
----------
 백신 접종 후 돌파감염 발생률은 백신 종류마다 다르다.
----------
 가명정보 활용 촉진을 위한 8개 과제 구체화, 26개 추진과제 마련 1. 가명정보 활용 촉진을 위한 8개 과제 구체화 1) 결합전문기관 기능 강화: 결합신청 접수, 결합키 생성, 결합데이터 반출 등 결합업무 수행 2) 가명정보 결합 절차 및 기준 명확화: 결합신청 접수, 결합키 생성, 결합데이터 반출 등 결합업무 수행 3) 결합데이터 반출 절차 간소화: 결합신청 접수, 결합키 생성, 결합데이터 반출 등
----------
 추미애 법무부 장관과 박원순 서울시장 등 범여권 중심의신천지 강제수사 주장에 대해 중앙재난안전대책본부(중대본)가 공식적으로 반대 입장을 밝혔다.
 예방접종 후 15~30분간은 접종기관에 머무시면서 이상반응 발생 여부를 관찰하시고 귀가 후에도 적어도 3시간 정도는 주의 깊게 관찰하시기 바랍니다. 발열, 피로감, 두통, 근육통, 메스꺼움, 구토와 같은 증상이 있으면 바로 의사의 진료를 받으시기 바랍니다.
----------
 2

In [ ]:
test_df = pd.read_csv(test_path)
test_df['summary'] = preds

In [ ]:
# 현재 시간으로 이름붙인 제출파일을 생성합니다.
TIME_SERIAL = datetime.now(timezone(timedelta(hours=9))).strftime("%y%m%d-%H%M%S")
SUBMISSION_PATH = f'/content/drive/MyDrive/공모전/GPT_Competition/exp_{TIME_SERIAL}.csv'
test_df[['id', 'summary']].to_csv(SUBMISSION_PATH, index=False)
print(SUBMISSION_PATH)

In [ ]:
# 자동으로 세션을 종료하고 싶을때 사용하세요.
from google.colab import runtime
runtime.unassign()